In [125]:
import pandas as pd
import numpy as np
import csv

In [126]:
for y in ["1996-97","1997-98","1998-99","1999-00","2000-01","2001-02","2002-03","2003-04","2004-05","2005-06","2006-07","2007-08","2008-09","2009-10","2010-11","2011-12","2012-13","2013-14","2014-15","2015-16","2016-17","2017-18"]:
    
    data = pd.read_csv(str("../ALL BASKETBALL/drive-data/csv/raw_shot_data/raw_shot_data_"+y+".csv"))
    data1 = data[["gameId","playerName","actionType", "shotDistance","shotMadeFlag"]]
    plays_dict = {}
    plays = set(data1["actionType"].values.tolist())
#     for play in plays:
#         if ("Hook" in play) | ("Turnaround" in play):
#             plays_dict[play]="Post"
#         elif ("Tip" in play) | ("Putback" in play) | ("Alley" in play):
#             plays_dict[play]="Reb/Oop"
#         elif "Driving" in play:
#             plays_dict[play]="Drive"
#         elif "Cutting" in play:
#             plays_dict[play]="Cut"
#         elif "Pull" in play:
#             plays_dict[play]="Pullup"
#         elif ("Layup" in play) | ("Finger" in play):
#             plays_dict[play]="Layup"
#         elif "Dunk" in play:
#             plays_dict[play]="Dunk"
#         elif ("Jump" in play) | ("Fadeaway" in play) | ("Stepback" in play):
#             plays_dict[play]="Jumper"
#         elif ("Running" in play):
#             plays_dict[play]="Layup"
#         else:
#             plays_dict[play]="invalid"      
#     data1["ShotClass"] = data1["actionType"].map(plays_dict)
    data1["ShotRange"] = np.where(data1["shotDistance"]>46, "47-94 Backcourt", 
                                 np.where(data1["shotDistance"]>34, "35-46 Near Half", 
                                         np.where(data1["shotDistance"]>27, "28-34 Deep", 
                                                  np.where(data1["shotDistance"]>23, "24-27 Three", 
                                                          np.where(data1["shotDistance"]>17, "18-23 Long Jumper", 
                                                                  np.where(data1["shotDistance"]>12, "13-17 Mid Range", 
                                                                          np.where(data1["shotDistance"]>8, "09-12 Short Range", 
                                                                                  np.where(data1["shotDistance"]>2, "03-08 Near Rim", 
                                                                                          "00-02 At Rim"))))))))
    shots = data1[["gameId","playerName","shotMadeFlag","ShotRange"]]#,"ShotClass"
    #shots.to_csv(str("shot_data/all_shots_"+y+".csv"),index=False,encoding="utf-8")
    shot_summ_df = shots.groupby(["playerName","ShotRange","shotMadeFlag"]).size().reset_index()#,"ShotClass"
    shot_summ_df.columns=["playername","ShotRange","Outcome","FGA"]#,"ShotClass"
    misses = shot_summ_df.loc[shot_summ_df["Outcome"]==0]
    misses = misses.rename(columns={"FGA":"Missed"}).reset_index(drop=True)
    makes = shot_summ_df.loc[shot_summ_df["Outcome"]==1]
    makes = makes.rename(columns={"FGA":"Made"}).reset_index(drop=True)
    shot_log = pd.merge(makes,misses,how="outer",on=["playername","ShotRange"])#,"ShotClass"
    shot_log = shot_log.fillna(0).drop(columns=["Outcome_x","Outcome_y"])
    shot_log[["Made","Missed"]] = shot_log[["Made","Missed"]].astype(int)
    shot_log["ShotPts"] = np.where(shot_log["ShotRange"].isin(["47-94 Backcourt","35-46 Near Half","28-34 Deep","24-27 Three"]),3,2)
    shot_log["FGA"]=shot_log["Made"] + shot_log["Missed"]
    shot_log["PPS"]=shot_log["ShotPts"] * (shot_log["Made"]/shot_log["FGA"])
    shot_log["Points"]=shot_log["Made"] * shot_log["ShotPts"]
    shot_log = shot_log.sort_values(["playername","ShotRange"])#,"ShotClass"
    shot_log = shot_log.drop(columns=["Made","Missed","FGA","ShotPts"]).reset_index(drop=True)
    #shot_log.to_csv(str("shot_data/shot_log_"+y+".csv"),index=False,encoding="utf-8")
    shot_pivot = pd.DataFrame(shot_log.pivot_table(values=["PPS","Points"],index=["playername"],columns=["ShotRange"])).fillna(0)#,"ShotClass"
    shot_pivot.columns = ['_'.join(col).rstrip() for col in shot_pivot.columns.values]
    shot_pivot.to_csv(str("shot_charts/shot_chart"+y+".csv"),encoding="utf-8")

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [127]:
for y in ["1996-97","1997-98","1998-99","1999-00","2000-01","2001-02","2002-03","2003-04","2004-05","2005-06","2006-07","2007-08","2008-09","2009-10","2010-11","2011-12","2012-13","2013-14","2014-15","2015-16","2016-17","2017-18"]:
    if y=="1996-97":
        dfdf = pd.read_csv(str("shot_charts/shot_chart"+y+".csv"))
        dfdf["Year"] = y[:2]+y[-2:]
    elif y=="1999-00":
        tempdf = pd.read_csv(str("shot_charts/shot_chart"+y+".csv"))
        tempdf["Year"] = "2000"
        dfdf = pd.concat([dfdf,tempdf])        
    else:
        tempdf = pd.read_csv(str("shot_charts/shot_chart"+y+".csv"))
        tempdf["Year"] = y[:2]+y[-2:]
        dfdf = pd.concat([dfdf,tempdf])

ddd=dfdf.set_index(["playername","Year"]).fillna(0)
ddd.to_csv("shot_charts/all_shots.csv",encoding="utf-8")

In [128]:
past = pd.read_csv("data/past_seasons_totals.csv").drop(columns=["Unnamed: 0","Unnamed: 0.1"])
sd = ddd.reset_index()
sd["Year"]=sd["Year"].astype(float)
past["Player"] = past["Player"].str.replace(".","").str.replace("*","").str.replace(r"'\'","^")
past["Player"] = past["Player"].str.split('^').str[0]
sd

,playername,Year,PPS_00-02 At Rim,PPS_03-08 Near Rim,PPS_09-12 Short Range,PPS_13-17 Mid Range,PPS_18-23 Long Jumper,PPS_24-27 Three,PPS_28-34 Deep,PPS_35-46 Near Half,PPS_47-94 Backcourt,Points_00-02 At Rim,Points_03-08 Near Rim,Points_09-12 Short Range,Points_13-17 Mid Range,Points_18-23 Long Jumper,Points_24-27 Three,Points_28-34 Deep,Points_35-46 Near Half,Points_47-94 Backcourt
0,A.C. Green,1997.0,1.162996,0.764706,0.916667,0.839695,0.724138,0.750000,0.000000,0.0,0.75,264.0,26.0,22.0,110.0,42.0,3.0,0.0,0.0,3.0
1,Aaron McKie,1997.0,1.126984,0.357143,0.166667,0.701754,0.854545,1.071429,0.000000,0.0,0.00,142.0,10.0,4.0,40.0,94.0,15.0,0.0,0.0,0.0
2,Aaron Williams,1997.0,1.339450,0.833333,0.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.00,146.0,20.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
3,Acie Earl,1997.0,0.862069,0.958333,0.731707,0.320000,0.000000,0.000000,0.000000,0.0,0.00,50.0,46.0,30.0,8.0,0.0,0.0,0.0,0.0,0.0
4,Adam Keefe,1997.0,1.134615,0.666667,0.909091,0.909091,0.857143,0.000000,0.000000,0.0,0.00,118.0,10.0,10.0,20.0,6.0,0.0,0.0,0.0,0.0
5,Adrian Caldwell,1997.0,1.057143,0.363636,0.000000,0.666667,0.000000,0.000000,0.000000,0.0,0.00,74.0,4.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
6,Alan Henderson,1997.0,1.203883,1.000000,0.476190,0.320000,0.666667,0.000000,0.000000,0.0,0.00,124.0,10.0,10.0,8.0,2.0,0.0,0.0,0.0,0.0
7,Allan Houston,1997.0,0.936170,0.968750,0.813187,0.819048,0.854167,1.010870,0.000000,0.0,0.00,176.0,62.0,74.0,172.0,328.0,93.0,0.0,0.0,0.0
8,Allen Iverson,1997.0,0.956386,0.988235,0.628571,0.738636,0.880000,0.842553,0.400000,0.0,0.00,614.0,84.0,44.0,130.0,242.0,198.0,6.0,0.0,0.0
9,Alonzo Mourning,1997.0,1.307292,0.931624,0.682927,1.068966,0.692308,0.000000,0.000000,0.0,0.00,502.0,218.0,84.0,124.0,18.0,0.0,0.0,0.0,0.0


In [129]:
clean_list = {}
for name in sorted(set(sd["playername"].astype(str))):
    inputname=name
    name=name.replace("Luc Mbah a Moute","LucRichardMbahAMoute")
    name=name.replace("Luc Richard Mbah a Moute","LucRichardMbahAMoute")
    name=name.replace("Luc Mbah","LucRichardMbahAMoute")
    name=name.replace("Nene","Nene Hilario")
    name=name.replace("Didier Ilunga-Mbenga","DJ Mbenga")
    name=name.replace("Nazareth","Naz")
    name=name.replace("Raulzinho","Raul")
    name=name.replace("James Webb III","James Webb")
    name=name.replace("James Web III","James Webb")
    name=name.replace("Juancho","Juan")
    name=name.replace("Patty Mills","Patrick Mills")
    name=name.replace("J.J. Barea","Jose Barea")
    name=name.replace("Ish Smith","Ishmael Smith")
    name=name.replace("Poeltl","Poltl")
    name=name.replace("Gigi","Luigi")
    name=name.replace("Vincent Hunter","Vince Hunter")
    name=name.replace("Joe Young","Joseph Young")
    name=name.replace("Maurice Harkless","Moe Harkless")
    name=name.replace("Wes Iwundu","Wesley Iwundu")
    name=name.replace("Wesley Johnson","Wes Johnson")
    name=name.replace("Metta World","Metta World Peace")
    name=name.replace("Taurean Waller-Prince","Taurean Prince")
    name=name.replace("Walter Lemon","Walt Lemon")
    for x in "ÀÁÂÃÄÅĀ":
        name=name.replace(x,"A");
    for x in "àáâãäåā":
        name=name.replace(x,"a");
    for x in "ëèéê":
        name=name.replace(x,"e");
    for x in "ôóøŏöò":
        name=name.replace(x,"o");
    for x in "úùüū":
        name=name.replace(x,"u");    
    name=name.replace("ģ","g")
    name=name.replace("č","c")
    name=name.replace("ć","c")
    name=name.replace("š","s")
    name=name.replace("ņ","n")
    name=name.replace("ñ","n")
    name=name.replace("í","i")
    name=name.replace("Ž","Z")
    name=name.replace("ž","z")
    name=name.replace("ý","y")
    name=name.replace("'","")
    name=name.replace("’","")
    name=name.replace(".","")
    name=name.replace(" Jr","")
    name=name.replace("IV","")
    name=name.replace("III","")
    name=name.replace("II","")
    name=name.replace("-","")
    name=name.replace(",","")
    name=name.replace(" ","")
    name=name.replace("*","")  
    clean_list[inputname] = name
sd["formatname"] = sd["playername"].astype(str).map(clean_list)
join = pd.merge(past,sd,how="left",on=["formatname","Year"])
#join = join.drop(columns=["Points_invalid_00-02 At Rim","Points_invalid_03-08 Near Rim","Points_invalid_09-12 Short Range","Points_invalid_13-17 Mid Range","Points_invalid_18-23 Long Jumper"])
join.fillna("").to_csv("data/tot_adv_shooting.csv",index=False,encoding="utf-8")